In [ ]:
import numpy as np
import pandas as pd
import wandb

api = wandb.Api()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager

# Set global figure background color
plt.style.use('seaborn-whitegrid')
plt.rcParams["figure.facecolor"] = "white"
sns.set(style="white")

# 한글 폰트 설정
font_name = font_manager.FontProperties(
    fname="/usr/share/fonts/NanumFont/NanumGothicBold.ttf"
).get_name()
plt.rc("font", family=font_name)
plt.rcParams["axes.unicode_minus"] = False

# Figure | 멀티 태스크 모델별 얼굴 검증 데이터셋 정확도

In [ ]:
sorted_columns = [
    # ("Basic", "a/accuracy"),
    # ("Basic", "ac/accuracy"),
    ("Basic", "f/accuracy"),
    # ("Basic", "fc/accuracy"),
    # ("Basic", "fn/accuracy"),
    # ("Basic", "g/accuracy"),
    # ("Basic", "gc/accuracy"),
    ("Family", "a/accuracy"),
    # ("Family", "ca/accuracy"),
    ("Family", "g/accuracy"),
    # ("Family", "cg/accuracy"),
    ("Family", "ag/accuracy"),
    # ("Family", "cag/accuracy"),
    # ("Personal", "a/accuracy"),
    ("Personal", "ac/accuracy"),
]

sorted_index = [
    "single-fr-ver-1-230529_0140-230602_1942",
    "dual-frkr-ver-1-230602_2021-230606_2104",
    "triple-fraekr-ver-1-230601_1838-230602_2112",
]

In [ ]:
# Project is specified by <entity/project-name>
runs = api.runs("jongphago/test_validation")

In [ ]:
summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

In [ ]:
runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list}
)

runs_df.to_csv("project.csv")
runs_df.drop(2, inplace=True)

In [ ]:
_summary_df = pd.DataFrame(
    {name: summary for summary, name in zip(runs_df.summary, runs_df.name)}
).T
_summary_df = _summary_df[
    sorted(_summary_df.columns[~_summary_df.columns.str.startswith("_")])
]

# 컬럼 이름을 분리하여 멀티 인덱스로 설정
_summary_df.columns = pd.MultiIndex.from_tuples(
    [tuple(c.split("-")) for c in _summary_df.columns]
)

summary_df = _summary_df.loc[sorted_index, sorted_columns].T.copy()
summary_df = summary_df.astype(float, copy=True)

In [ ]:
summary_df.mean()

In [ ]:
temp = summary_df.copy()
temp.columns = temp.columns.str.split('-').map(lambda x: x[0])
temp.iloc[:, :1]

In [ ]:
diff_index = 1, 0  # Dual, single
diff_index = 2, 0  # Triple, single
acc_df1, acc_df2 = summary_df.T.iloc[diff_index[0]],  summary_df.T.iloc[diff_index[1]]
name1, name2 = acc_df1.name.split('-')[0], acc_df2.name.split('-')[0]
diff_pair = np.subtract(acc_df1, acc_df2)
acc_diff_df = diff_pair.to_frame(
    name="Accuracy difference"
)

# 히트맵 생성
sns.heatmap(
    acc_diff_df,
    annot=True,
    cmap="coolwarm",
    fmt=".4g",
    cbar_kws={"label": "difference"},
)

print(f"Figure | {name1.upper()}과 {name2.upper()}의 얼굴 검증 데이터셋 정확도 차이 비교")


# 타이틀과 라벨을 설정합니다.
plt.title(f"{name1.upper()}과 {name2.upper()}의 얼굴 검증 데이터셋 정확도 차이 비교", fontsize=13)
plt.xlabel("모델간 정확도 차이", fontsize=10)
plt.ylabel("얼굴 검증 데이터셋", fontsize=10)


# 그래프를 출력합니다.
plt.show()

In [ ]:
acc_diff_df.groupby(level=0).mean()

In [ ]:
# 컬럼명에서 첫 번째 원소를 추출하여 X축 라벨로 사용합니다.
edited_x_label = summary_df.columns.str.rsplit("-", 4).map(lambda x: x[0].upper())

# 그림 크기를 설정하고, 히트맵을 생성합니다.
fig, ax = plt.subplots(figsize=(10, 8))

# 히트맵 생성
sns.heatmap(
    summary_df,
    annot=True,
    cmap="coolwarm",
    fmt=".4g",
    cbar_kws={"label": "Accuracy"},
    xticklabels=edited_x_label,
    ax=ax,
)
print("Figure | 멀티 태스크 모델별 얼굴 검증 데이터셋 정확도")

# 타이틀과 라벨을 설정합니다.
ax.set_title("멀티 태스크 모델별 얼굴 검증 데이터셋 정확도", fontsize=15)
ax.set_xlabel("얼굴 검증 데이터셋 종류", fontsize=12)
ax.set_ylabel("멀티 태스크 모델 종류", fontsize=12)

# x, y 축 라벨의 회전 각도를 설정합니다.
plt.xticks(rotation=45)
# plt.yticks(rotation=45)

# 그래프를 출력합니다.
plt.show()


# Figure | Single Model - Grouped pair - Accuracy

In [ ]:
import os
from itertools import product
task_group_pairs = product([1, 2, 3], ['basic', 'family', 'personal', 'total'])
for num_task, selected_pair_group in task_group_pairs:
    # selected pair group only
    if selected_pair_group != 'total':
        is_pair_group = summary_df.index.map(lambda x: x[0]) == selected_pair_group.capitalize()
        selected_index = summary_df.index[is_pair_group]
    else:
        selected_index = summary_df.index

    # 컬럼명에서 첫 번째 원소를 추출하여 X축 라벨로 사용합니다.
    edited_x_label = summary_df.columns.str.rsplit("-", 4).map(lambda x: x[0].upper())[
        num_task - 1
    ]

    # 그림 크기를 설정하고, 히트맵을 생성합니다.
    fig, ax = plt.subplots(figsize=(10, 8))

    # 히트맵 생성
    sns.heatmap(
        summary_df.loc[selected_index].iloc[:, num_task - 1 : num_task],
        annot=True,
        cmap="coolwarm",
        fmt=".4g",
        cbar_kws={"label": "Accuracy"},
        xticklabels=[edited_x_label],
        vmin=0.5, vmax=1.0,
        ax=ax,
    )
    print("Figure | 멀티 태스크 모델별 얼굴 검증 데이터셋 정확도")

    # 타이틀과 라벨을 설정합니다.
    ax.set_title(f"{edited_x_label} 얼굴 검증 데이터셋 정확도", fontsize=15)
    ax.set_ylabel("얼굴 검증 데이터셋", fontsize=12)
    ax.set_xlabel("멀티 태스크 모델", fontsize=12)

    # x, y 축 라벨의 회전 각도를 설정합니다.
    # plt.xticks(rotation=45)
    # plt.yticks(rotation=45)

    # 그래프를 출력합니다.{
    plt.tight_layout()
    os.makedirs("output/accuracy/selected", exist_ok=True)
    # plt.savefig(f'output/accuracy/selected/{edited_x_label}-{selected_pair_group}.png', facecolor='w')
    plt.show()
    # break

# Figure | Double Model

In [ ]:
from itertools import product

task_group_pairs = product([(1, 2), (1, 3)], ["basic", "family", "personal", "total"])
for num_task, selected_pair_group in task_group_pairs:
    # selected pair group only
    if selected_pair_group != "total":
        is_pair_group = (
            summary_df.index.map(lambda x: x[0]) == selected_pair_group.capitalize()
        )
        selected_index = summary_df.index[is_pair_group]
    else:
        selected_index = summary_df.index

    num_task1, num_task2 = num_task

    # 컬럼명에서 첫 번째 원소를 추출하여 X축 라벨로 사용합니다.
    edited_x_label = summary_df.columns.str.rsplit("-", 4).map(lambda x: x[0].upper())[
        [num_task1 - 1, num_task2 - 1]
    ]
    edited_x_label
    # 그림 크기를 설정하고, 히트맵을 생성합니다.
    fig, (ax1, ax2) = plt.subplots(
        figsize=(9, 6), ncols=2, gridspec_kw={"width_ratios": [3, 1]}
    )

    # 첫번째 히트맵 생성
    sns.heatmap(
        summary_df.loc[selected_index].iloc[:, [num_task1 - 1, num_task2 - 1]],
        annot=True,
        cmap="coolwarm",
        fmt=".4g",
        cbar_kws={"label": "Accuracy"},
        xticklabels=edited_x_label,
        vmin=0.5,
        vmax=1.0,
        ax=ax1,
    )

    # 타이틀과 라벨을 설정합니다.
    ax1.set_title(f"{'와 '.join(edited_x_label)}의 얼굴 검증 데이터셋 정확도", fontsize=15)
    ax1.set_ylabel("얼굴 검증 데이터셋", fontsize=12)
    ax1.set_xlabel("멀티 태스크 모델", fontsize=12)

    acc_df1, acc_df2 = (
        summary_df.loc[selected_index].T.iloc[num_task1 - 1],
        summary_df.loc[selected_index].T.iloc[num_task2 - 1],
    )
    name1, name2 = acc_df1.name.split("-")[0], acc_df2.name.split("-")[0]
    diff_pair = np.subtract(acc_df2, acc_df1)
    acc_diff_df = diff_pair.to_frame(name="Accuracy difference")

    # 두번째 히트맵 생성
    sns.heatmap(
        acc_diff_df,
        annot=True,
        cmap="PiYG",
        fmt=".4g",
        cbar_kws={"label": "difference"},
        vmin=-0.25,
        vmax=0.25,
        ax=ax2,
    )

    # 타이틀과 라벨을 설정합니다.
    ax2.set_title(f"얼굴 검증 정확도 차이", fontsize=13)
    ax2.set_xlabel("모델간 정확도 차이", fontsize=10)
    ax2.set_ylabel("")
    ax2.set_yticks([])

    # 그래프를 출력합니다.
    plt.tight_layout()
    os.makedirs("output/acc_diff/selected", exist_ok=True)
    # plt.savefig(f'output/acc_diff/selected/{"-".join(edited_x_label)}-{selected_pair_group}.png', facecolor='w')
    plt.show()
    break

In [ ]:
# 컬럼명에서 첫 번째 원소를 추출하여 X축 라벨로 사용합니다.
# edited_x_label = summary_df.columns.str.rsplit("-", 4).map(lambda x: x[0].upper())

# 그림 크기를 설정하고, 히트맵을 생성합니다.
fig, ax = plt.subplots(figsize=(10, 8))

# 히트맵 생성
sns.heatmap(
    summary_df,
    annot=True,
    cmap="coolwarm",
    fmt=".4g",
    cbar_kws={"label": "Accuracy"},
    # xticklabels=edited_x_label,
    ax=ax,
)
print("Figure | 멀티 태스크 모델별 얼굴 검증 데이터셋 정확도")

# 타이틀과 라벨을 설정합니다.
ax.set_title("멀티 태스크 모델별 얼굴 검증 데이터셋 정확도", fontsize=15)
ax.set_xlabel("얼굴 검증 데이터셋 종류", fontsize=12)
ax.set_ylabel("멀티 태스크 모델 종류", fontsize=12)

# x, y 축 라벨의 회전 각도를 설정합니다.
plt.xticks(rotation=45)
# plt.yticks(rotation=45)

# 그래프를 출력합니다.
plt.show()


# Sweep

## Single task

In [ ]:
import pandas as pd
import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("jongphago/single-fr-ver-1")
# runs = api.runs("jongphago/dual-frkr-ver-1")
# runs = api.runs("jongphago/triple-fraekr-ver-1")

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list}
)

runs_df.to_csv("project.csv")
# 'summary' 열을 flatten하고, 데이터프레임으로 변환합니다.
summary_df = pd.json_normalize(runs_df["summary"])

# 'config' 열을 flatten하고, 데이터프레임으로 변환합니다.
config_df = pd.json_normalize(runs_df["config"])

# 원래 데이터프레임에서 'summary'와 'config' 열을 제거합니다.
runs_df = runs_df.drop(["summary", "config"], axis=1)

# flatten된 'summary'와 'config' 데이터프레임을 원래 데이터프레임과 병합합니다.
runs_df = pd.concat([runs_df, summary_df, config_df], axis=1)

# Rename columns 
runs_df.columns = runs_df.columns.str.replace("/", "_").map(lambda x: x.lower())
runs_df['_timestamp'] = pd.to_datetime(runs_df['_timestamp'], unit='s')

In [ ]:
selected = runs_df[runs_df._timestamp <= "2023-05-31 04:25:28.001810176"]  # single
# selected = runs_df[~(runs_df.valid_accuracy.isnull())]  # dual
# selected = runs_df[runs_df._timestamp <= "2023-06-03 16:00:00.0"]  # triple
selected.shape

In [ ]:
config_columns = config_df.columns

In [ ]:
num_total = len(selected)
valid_max = f"{selected.valid_accuracy.sort_values(ascending=False).iloc[0]:.4f}"
test_max = f"{selected.test_accuracy.sort_values(ascending=False).iloc[0]:.4f}"
mean, std = (
    selected.valid_accuracy.mean(),
    selected.valid_accuracy.std(),
)  # (0.5874, 0.0955)
over_80 = (selected.valid_accuracy >= 0.8).sum()  # 4

print(f"- {num_total}회 실행 하여 최고 검증 정확도 {valid_max} 달성 (테스트 정확도: {test_max})")
print(f" - 총 {len(runs_df)}번의 시도 중 {num_total}번의 검증 정확도가 기록")
print(f" - 모델 최적화 과정에서 기록된 검증 정확도의 평균 {mean:.4f}, 표준편차 {std:.4f}")
print(f" - {num_total}회중 {over_80}번의 시도({over_80/num_total:.4f})가 검증정확도를 0.8이상을 기록")

In [ ]:
config_name = [
    "network",
    "embedding_size",
    "optimizer",
    "margin_list",
    "num_epoch",
    "lr",
    "weight_decay",
    "momentum",
    "dropout",
    "sample_rate",
    "face_weight",
    # "kinship_weight",
    # "age_weight",
    # "age_group_weight",
    # "age_mean_var_weight",
]
multi_config_name = pd.MultiIndex.from_tuples(
    [
        ("fixed", "network"),
        ("fixed", "embedding_size"),
        ("fixed", "optimizer"),
        ("fixed", "margin_list"),
        ("optimized", "num_epoch"),
        ("optimized", "lr"),
        ("optimized", "weight_decay"),
        ("optimized", "momentum"),
        ("optimized", "dropout"),
        ("optimized", "sample_rate"),
        ("weight(optim)", "face_weight"),
        # ("weight(optim)", "kinship_weight"),
        # ("weight(optim)", "age_weight"),
        # ("weight(optim)", "age_group_weight"),
        # ("weight(optim)", "age_mean_var_weight"),
    ]
)
print(len(config_name))

In [ ]:
max_index = selected.valid_accuracy.argmax()
params_df = selected.iloc[max_index][config_name].to_frame(name='value')
params_df.index = multi_config_name
params_df

In [ ]:
prop_name = "sample_rate"
selected.loc[:, "temp"] = pd.cut(selected[prop_name], bins=10)
qdf = selected.groupby("temp").valid_accuracy.mean().to_frame()
# 그림 크기를 설정하고, 히트맵을 생성합니다.
fig, ax = plt.subplots(figsize=(10, 8))
cdf = selected.groupby("temp").valid_accuracy.count().to_frame().T
display(cdf)
# 히트맵 생성
sns.heatmap(
    qdf,
    annot=True,
    cmap="coolwarm",
    fmt=".4g",
    cbar_kws={"label": "Accuracy"},
    ax=ax,
    vmin=0.5, vmax=1.0
)
plt.show()